In [2]:
import cudf
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [17]:
%%time 
transactions = cudf.read_csv('storage/transactions_train.csv')
articles = cudf.read_csv('storage/articles.csv')
customers = cudf.read_csv('storage/customers.csv')

CPU times: user 720 ms, sys: 1.03 s, total: 1.75 s
Wall time: 2.88 s


In [18]:
transactions['t_dat'] = cudf.to_datetime(transactions['t_dat'])
customers['FN'].fillna(0.,inplace=True)
customers['Active'].fillna(0.,inplace=True)
customers['club_member_status'].fillna('None',inplace=True)
customers['age'] = customers['age'] / 10
customers['age'] = customers['age'].astype(int)
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].str.lower().fillna('none')

In [19]:
transactions = transactions.merge(customers,on='customer_id',how='left')
#transactions = transactions.merge(articles,on='article_id',how='left')

In [20]:
customers = customers.merge(
    transactions.groupby('customer_id')['sales_channel_id'].mean().to_frame().reset_index().rename(columns={'sales_channel_id':'sales_channel_habit'}),
    on='customer_id',
    how='left',
)
customers['sales_channel_habit'] = (customers['sales_channel_habit'] > 1.5).astype(int)
customers['sales_channel_habit'].fillna(0.,inplace=True)

Index(['customer_id', 'FN', 'Active', 'club_member_status',
       'fashion_news_frequency', 'age', 'postal_code', 'sales_channel_habit'],
      dtype='object')


In [43]:
def construct_candidate_dict(transactions,key='customer_id'):
    purchase_dict = {}
    for i,x in enumerate(zip(transactions[key], transactions['article_id'])):
        key_id, art_id = x
        if key_id not in purchase_dict:
            purchase_dict[key_id] = {}
        if art_id not in purchase_dict[key_id]:
            purchase_dict[key_id][art_id] = 0
        purchase_dict[key_id][art_id] += 1
    return purchase_dict

def make_customer_demography_dict(transactions):
    key = ['age','fashion_news_frequency','club_member_status','article_id']
    tmp = transactions.groupby(key).size().to_frame('count').reset_index()
    values = transactions.groupby(key).size().index.unique().tolist()
    d = {}
    #for a,f,c in values:
    for a in values:
        #arr = tmp[(tmp['age']==a)&(tmp['fashion_news_frequency']==f)&(tmp['club_member_status']==c)]['article_id']
        arr = tmp[tmp['age']==a]['article_id']
        item = arr.sort_values(ascending=False).tolist()[:12]
        d[a,f,c] = item
    return d

def construct_pred_df(
        test_df,transactions,
        nweek=3,
    ):
    
    recent_transactions = {}
    purchase_dict = {}
    for i in range(1,nweek+1):
        recent_transactions[i] = transactions[(transactions.t_dat>transactions.t_dat.max()-i*pd.Timedelta(7,unit='day'))&(transactions.t_dat<=transactions.t_dat.max()-(i-1)*pd.Timedelta(7,unit='day'))]
        purchase_dict[i] = construct_candidate_dict(recent_transactions[i])
    

    most_popular_items_1w_sch1 = list((recent_transactions[1][recent_transactions[1]['sales_channel_id']==1]['article_id'].value_counts()).index)[:12]
    most_popular_items_1w_sch2 = list((recent_transactions[1][recent_transactions[1]['sales_channel_id']==2]['article_id'].value_counts()).index)[:12]
    most_popular_items_1w_all = list((recent_transactions[1]['article_id'].value_counts()).index)[:12]

    pred_df = test_df[['customer_id']]
    prediction_list = []
    
    for i, (cust_id,sales_channel_habit) in enumerate(
        zip(
            test_df['customer_id'].values.reshape((-1,)),
            test_df['sales_channel_habit'].values.reshape((-1,)),
        )
    ):
        s = []
        total_purchase_dict = {}
        
        for i,purchase_dict_week in purchase_dict.items():
            if cust_id in purchase_dict_week:
                l = sorted((purchase_dict_week[cust_id]).items(), key=lambda x: x[1], reverse=True)
                l = [y[0] for y in l][:12]
                for aid in l:
                    if aid not in total_purchase_dict:
                        total_purchase_dict[aid] = 1
                    else:
                        total_purchase_dict[aid] += 1

        if sales_channel_habit == 1:
            most_popular_items_1w = most_popular_items_1w_sch1
        else:
            most_popular_items_1w = most_popular_items_1w_sch2
        
        most_popular_items_1w = most_popular_items_1w_all

        for aid in most_popular_items_1w[:12]:
            if aid not in total_purchase_dict:
                total_purchase_dict[aid] = 1
            else:
                total_purchase_dict[aid] += 1

        total_purchase_dict = {k: v for k, v in sorted(total_purchase_dict.items(), key=lambda item: item[1], reverse=True)}
        s = list(total_purchase_dict.keys())[:12]
        
        prediction_list.append(s)
        
    pred_df['prediction'] = prediction_list
    return pred_df

def construct_gt_df(test_transactions):
    gt_df = test_transactions.to_pandas().groupby('customer_id')['article_id'].agg(lambda x: x.tolist()).reset_index()
    gt_df.columns = ['customer_id','ground_truth']
    return gt_df

def evaluate_score(pred_df,gt_df,k=12,verbose=True,group_name='customer_id'):
    from metric import mapk
    eval_df = gt_df.merge(pred_df,on=group_name,how='left')
    score = mapk(eval_df['ground_truth'].tolist(),eval_df['prediction'].tolist())
    if verbose: print('map@'+str(k),score)
    return score

In [45]:
max_score = -1
for _ in range(1):
    scores = []
    for i,(trn_start_time,trn_end_time,test_start_time,test_end_time) in enumerate([
            ('2020-01-01','2020-09-01','2020-09-01','2020-09-07'),
            ('2020-01-01','2020-09-07','2020-09-07','2020-09-15'),
            ('2020-01-01','2020-09-15','2020-09-15','2020-09-22'),
        ]):
        trn_transactions = transactions[(transactions.t_dat > cudf.to_datetime(trn_start_time)) & (transactions.t_dat <= cudf.to_datetime(trn_end_time))]
        test_transactions = transactions[(transactions.t_dat > cudf.to_datetime(test_start_time)) & (transactions.t_dat <= cudf.to_datetime(test_end_time))]
        gt_df = construct_gt_df(test_transactions)
        test_df = test_transactions[['customer_id']].drop_duplicates()
        for f in ['sales_channel_habit']:
            test_df = test_df.merge(customers[['customer_id',f]],on='customer_id',how='left')

        pred_df = construct_pred_df(test_df.to_pandas(),trn_transactions.to_pandas())
        score = evaluate_score(
            pred_df,
            gt_df,
            verbose=False,
        )
        scores.append(score)
    mean_score = np.mean(scores)
    print(mean_score,scores)
    if np.mean(scores) > max_score:
        max_score = np.mean(scores)
        best_scores = scores
print('Best score',max_score,best_scores)

0.021557291709346407 [0.02146737896690352, 0.019980594092469595, 0.023223902068666105]
Best score 0.021557291709346407 [0.02146737896690352, 0.019980594092469595, 0.023223902068666105]


****Submission****

In [166]:
out_dir = 'storage/output/220320_candidate_generation/'

In [167]:
submission_df = pd.read_csv('storage/sample_submission.csv')
pred_df = construct_pred_df(submission_df,transactions.to_pandas())

In [168]:
pred_df['prediction'] = pred_df['prediction'].apply(lambda x: ' '.join(['0'+str(i) for i in x]))

In [169]:
os.makedirs(out_dir,exist_ok=True)
pred_df.to_csv(os.path.join(out_dir,'submission.csv'),index=False)

In [38]:
pred_df

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0924243001 0924243002 0918522001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0924243002 0918522001 0923758001 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0924243001 0924243002 0918522001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0924243002 0918522001 0923758001 08...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0924243001 0924243002 0918522001 0923758001 08...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0713997002 0720125039 0740922009 0791587007 08...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0924243001 0924243002 0918522001 0923758001 08...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0762846027 0689365050 0884081001 0794819001 09...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0924243001 0924243002 0918522001 0923758001 08...


****Candidate analysis****

In [35]:
pred_df = pred_df.merge(gt_df,on='customer_id')

In [41]:
pred_df['n'] = pred_df['prediction'].apply(len)
pred_df['intersection'] = pred_df.apply(lambda x: set(x['prediction']).intersection(x['ground_truth']),axis=1)
pred_df['n_intersection'] = pred_df['intersection'].apply(len)

In [51]:
tmp = pred_df.loc[pred_df['n_intersection']==0]['ground_truth'].explode('ground_truth').value_counts()
tmp = cudf.from_pandas(tmp)

In [52]:
df_nopred = tmp.to_frame().reset_index().rename(columns={'index':'article_id','ground_truth':'count'}).merge(articles,on='article_id',how='left')

In [ ]:
df_nopred.groupby('product_code')['count'].sum().sort_values(ascending=False).iloc[:50]

In [56]:
df_nopred.columns

Index(['article_id', 'count', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [234]:
df_nopred = pred_df.loc[pred_df['n']==0]['ground_truth'].explode('ground_truth').value_counts()

In [231]:
max_dat = transactions.groupby('article_id')['t_dat'].max().to_frame().reset_index()
min_dat = transactions.groupby('article_id')['t_dat'].min().to_frame().reset_index()

In [236]:
df_nopred = df_nopred.to_frame().reset_index().rename(columns={'index':'article_id'}).merge(max_dat.to_pandas(),on='article_id',how='left')
df_nopred = df_nopred.merge(min_dat.to_pandas(),on='article_id',how='left')

In [62]:
pred_df

,customer_id,prediction,ground_truth,n,intersection,n_intersection
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[909370001, 865799006, 918522001, 448509014, 7...",[794321007],12,{},0
1,00039306476aaf41a07fed942884f16b30abfa83a2a8be...,"[909370001, 865799006, 918522001, 448509014, 7...",[624486001],12,{},0
2,00040239317e877c77ac6e79df42eb2633ad38fcac09fc...,"[875272011, 875272012, 909370001, 865799006, 9...","[875272011, 875272012]",12,"{875272011, 875272012}",2
3,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,"[399136061, 732842014, 556255001, 852219003, 7...","[757926001, 788575004, 640021019]",12,{},0
4,000749135ee9aa3a24c2316ea5ae4f495b39c1653c5612...,"[909370001, 865799006, 918522001, 448509014, 7...","[800691007, 800691007, 800691008, 800691008]",12,{},0
...,...,...,...,...,...,...
67139,fff98edc27fc5d64c3027bf0e3702510143d1a79c3dc9a...,"[909370001, 865799006, 918522001, 448509014, 7...","[898918002, 855198005, 902163001, 913688001, 8...",12,{},0
67140,fffa67737587e52ff1afa9c7c6490b5eb7acbc439fe82b...,"[909370001, 865799006, 918522001, 448509014, 7...","[874816003, 911870004]",12,{},0
67141,fffa7d7799eb390a76308454cbdd76e473d65b1497fbe4...,"[909370001, 865799006, 918522001, 448509014, 7...","[861803014, 849886010]",12,{},0
67142,fffae8eb3a282d8c43c77dd2ca0621703b71e90904dfde...,"[914441004, 914441005, 881244001, 882612004, 4...","[396135007, 817472007, 715624050, 817472003, 8...",12,{797892001},1


****Purchase rate**** (No use, performance dropped)

In [12]:
min_dat_purchase = transactions.groupby(['article_id'])['t_dat'].min()
max_dat_purchase = transactions.groupby(['article_id'])['t_dat'].max()
sale_duration = (max_dat_purchase - min_dat_purchase).to_frame().reset_index().rename(columns={'t_dat':'duration'})
sale_duration['duration'] = sale_duration['duration'].dt.days
sale_count = transactions.groupby(['article_id'])['t_dat'].count().to_frame().reset_index().rename(columns={'t_dat':'count'})

In [20]:
sale_rate = sale_duration.merge(sale_count,on='article_id')
sale_rate = sale_rate.loc[sale_rate['duration']!=0]

In [21]:
sale_rate['rate'] = sale_rate['count'] / sale_rate['duration']

In [31]:
sale_rate.sort_values('rate',ascending=False)['article_id'].to_arrow().to_pylist()[:12]

[934835001,
 909370001,
 918522001,
 706016001,
 923758001,
 751471043,
 924243001,
 889550002,
 935541001,
 918292001,
 916468003,
 898694001]

****Number of customer****

In [9]:
tmp = transactions.groupby('article_id')['customer_id'].nunique().to_frame().reset_index()
print(tmp['article_id'].iloc[:12].to_arrow().to_pylist())

[108775015, 108775044, 108775051, 110065001, 110065002, 110065011, 111565001, 111565003, 111586001, 111593001, 111609001, 112679048]


****Age analysis****

In [44]:
transactions['age'] = transactions['age'].astype(int)
tmp = transactions.groupby(['age','article_id']).size().to_frame('count').reset_index()
for a in range(1,10):
    item = tmp.loc[tmp['age']==a].sort_values('count',ascending=False)['article_id'].to_arrow().to_pylist()[:20]
    print(a,,item)

1 [706016001, 759871002, 706016002, 464297007, 759871001, 372860002, 673396002, 720125001, 733749001, 759465001, 448509014, 372860001, 685814003, 706016003, 684209004, 688537004, 706016006, 684209013, 706016015, 741356002]
2 [706016001, 706016002, 372860001, 759871002, 156231001, 464297007, 610776002, 448509014, 399223001, 706016003, 372860002, 720125001, 673396002, 610776001, 351484002, 688537004, 562245046, 158340001, 590928001, 673677002]
3 [610776002, 706016001, 464297007, 372860001, 562245001, 160442007, 158340001, 610776001, 608776002, 562245046, 351484002, 706016002, 565379001, 111593001, 720125001, 717490008, 111586001, 562245018, 399256001, 590928001]
4 [706016001, 706016002, 372860001, 610776002, 372860002, 568601006, 399256001, 351484002, 572797001, 562245001, 673677002, 562245046, 572797002, 610776001, 579541001, 608776002, 751471001, 673396002, 573716012, 706016003]
5 [706016001, 706016002, 372860001, 568601006, 372860002, 610776002, 751471001, 673677002, 678942001, 579541

In [47]:
tmp = transactions.groupby(['fashion_news_frequency','article_id']).size().to_frame('count').reset_index()
values = transactions['fashion_news_frequency'].unique().to_arrow().to_pylist()
for a in values:
    item = tmp.loc[tmp['fashion_news_frequency']==a].sort_values('count',ascending=False)['article_id'].to_arrow().to_pylist()[:20]
    print(a,item)

monthly [372860001, 565379001, 638553001, 160442007, 399223001, 111593001, 673677002, 706016001, 688537004, 703366002, 562245018, 351484002, 464297007, 590928001, 160442010, 507909001, 608776002, 759871002, 372860002, 556539003]
none [706016001, 706016002, 372860001, 610776002, 464297007, 759871002, 372860002, 610776001, 351484002, 399223001, 720125001, 156231001, 706016003, 562245046, 562245001, 568601006, 448509014, 160442007, 688537004, 608776002]
regularly [706016001, 706016002, 372860001, 610776002, 759871002, 372860002, 464297007, 399223001, 399256001, 610776001, 562245001, 706016003, 673396002, 673677002, 562245046, 156231001, 720125001, 751471001, 448509014, 568601006]


In [104]:
tmp = transactions.groupby(['club_member_status','article_id']).size().to_frame('count').reset_index()
values = transactions['club_member_status'].unique().to_arrow().to_pylist()
for a in values:
    item = tmp.loc[tmp['club_member_status']==a].sort_values('count',ascending=False)['article_id'].to_arrow().to_pylist()[:5]
    print(a,item)

ACTIVE [706016001, 706016002, 372860001, 610776002, 759871002]
LEFT CLUB [720137001, 355072001, 457892003, 484398001, 448509001]
None [706016001, 706016002, 610776001, 706016003, 562245001]
PRE-CREATE [706016001, 706016002, 351484002, 610776002, 562245001]


In [135]:
tmp = transactions.groupby(['age','fashion_news_frequency','club_member_status','article_id']).size().to_frame('count').reset_index()
values = transactions.groupby(['age','fashion_news_frequency','club_member_status']).size().index.unique().to_arrow()
d = {}
for ai,fi,ci in zip(*values):
    a,f,c = ai.as_py(),fi.as_py(),ci.as_py()
    arr = tmp[(tmp['age']==a)&(tmp['fashion_news_frequency']==f)&(tmp['club_member_status']==c)]['article_id']
    item = arr.sort_values(ascending=False).to_arrow().to_pylist()[:5]
    d[a,f,c] = item

****New items****

In [ ]:
transactions.loc[transactions['t_dat'] > cudf.to_datetime('2020-07-01')].groupby('article_id').size().sort_values(ascending=False).index.to_arrow().to_pylist()[:12]

****Sale channel****

In [246]:
print(transactions.loc[transactions['sales_channel_id']==1]['article_id'].value_counts().iloc[:20])
print(transactions.loc[transactions['sales_channel_id']==2]['article_id'].value_counts().iloc[:20])

372860001    22233
156231001    19494
372860002    15330
673677002    13334
111586001    13199
399256001    12879
608776002    12707
464297007    11963
399223001    11733
111593001    11410
158340001    10948
759871002    10282
228257001     8933
565379001     8351
610776002     8249
160442007     8106
706016001     7743
579541001     7667
507909001     7522
673396002     7397
Name: article_id, dtype: int32
706016001    42544
706016002    29292
610776002    21950
706016003    19881
562245001    19462
448509014    18665
351484002    17964
759871002    16047
610776001    15701
720125001    15549
590928001    15469
562245046    14712
741356002    14201
688537004    14145
484398001    13950
464297007    13062
573716012    12966
706016015    12909
751471001    12705
554450001    12689
Name: article_id, dtype: int32


In [253]:
transactions.groupby('customer_id')['sales_channel_id'].nunique().value_counts()

1    879026
2    483255
Name: sales_channel_id, dtype: int32

In [15]:
tmp = customers.merge(
    transactions.groupby('customer_id')['sales_channel_id'].mean().to_frame().reset_index().rename(columns={'sales_channel_id':'sales_channel_habit'}),
    on='customer_id',
    how='left',
)
tmp['sales_channel_habit'] = (tmp['sales_channel_habit'] > 1.5).astype(int)

In [16]:
tmp

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,sales_channel_habit
0,067beb11ab2f611c4c86c97fdb2750bcf51a5f27116ae0...,1.0,1.0,ACTIVE,regularly,3,c03f1add73fd1f27700939db782400e40c03fc822153b4...,1
1,067c0097961d9b22593daff46cab96cb6ed07f6ee1f1e4...,0.0,0.0,ACTIVE,none,2,b1bd8b73281f1f6a1b8cd6e996ee61e8c6eadd4730b75e...,1
2,067c08a7951052c6c282d065f2a054c42dc3543e14423b...,0.0,0.0,ACTIVE,none,2,3b04865d18813644d5f42a4691ac2a05e0a80ca0067a96...,1
3,067c0cfa74748bec0039dfd492b149908a2ec6a6d871ae...,0.0,0.0,PRE-CREATE,none,2,d1a03714cf80b5b6e29205096e30fc5069a832cb072099...,1
4,067c1776e91fa7a686b2d026129fb81c290deaf436816f...,1.0,1.0,ACTIVE,regularly,5,18b39b37b07dcd1485a0ed504e3e5cc19431c0e3323187...,1
...,...,...,...,...,...,...,...,...
1371975,ffedb855f86928ed0ced39dfa821d5a227ff8aefd9a372...,0.0,0.0,ACTIVE,none,2,c113e18ec7bf350503d806dd0ddaa49c244a30815224d6...,0
1371976,ffedc92bc0daea38ca7283ec2dd3b30f475766dbbe3fc4...,1.0,1.0,ACTIVE,regularly,5,2c29ae653a9282cce4151bd87643c907644e09541abc28...,0
1371977,ffedcaed36b1fdb1d6581a3d9dc77e68fac3d1ecd4eb4c...,0.0,0.0,ACTIVE,none,2,444af3fc241628212c762ed271bf8d373e840f24aa959f...,1
1371978,ffed85b312a57a1f36092fdf9ff03f2c07ba8425e1ef97...,0.0,0.0,ACTIVE,none,4,5f5f89db9486e060a1cadaa227f406c40ae9e7d1687c8e...,0


****Misc****

In [48]:
transactions

,t_dat,customer_id,article_id,price,sales_channel_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,2018-09-20,1c12579d3cda47a6d173dc2eb5c6d3cbeae4852b5a1d1c...,676698001,0.042356,2,1.0,1.0,ACTIVE,regularly,3,8f70f26427c735a8d8b83a10711e6df2d4956113eb6c83...
1,2018-09-20,1c12579d3cda47a6d173dc2eb5c6d3cbeae4852b5a1d1c...,717341001,0.067780,2,1.0,1.0,ACTIVE,regularly,3,8f70f26427c735a8d8b83a10711e6df2d4956113eb6c83...
2,2018-09-20,1c12579d3cda47a6d173dc2eb5c6d3cbeae4852b5a1d1c...,639835001,0.084729,2,1.0,1.0,ACTIVE,regularly,3,8f70f26427c735a8d8b83a10711e6df2d4956113eb6c83...
3,2018-09-20,1c12579d3cda47a6d173dc2eb5c6d3cbeae4852b5a1d1c...,594834001,0.011847,2,1.0,1.0,ACTIVE,regularly,3,8f70f26427c735a8d8b83a10711e6df2d4956113eb6c83...
4,2018-09-20,1c12579d3cda47a6d173dc2eb5c6d3cbeae4852b5a1d1c...,671505002,0.033881,2,1.0,1.0,ACTIVE,regularly,3,8f70f26427c735a8d8b83a10711e6df2d4956113eb6c83...
...,...,...,...,...,...,...,...,...,...,...,...
31788319,2020-09-22,f2a1a1b7252273dd3ed009a5f65f56ca98a9224e0af5ba...,928907001,0.050831,2,1.0,1.0,ACTIVE,regularly,2,ac4ea1f737ef462577cd0f780c5c73ab474eb64248bd57...
31788320,2020-09-22,f2a1a1b7252273dd3ed009a5f65f56ca98a9224e0af5ba...,921266006,0.016932,2,1.0,1.0,ACTIVE,regularly,2,ac4ea1f737ef462577cd0f780c5c73ab474eb64248bd57...
31788321,2020-09-22,f2b0779e6d6c712f296ab9362da347501228101e241da1...,870989001,0.101678,2,1.0,1.0,ACTIVE,regularly,2,3820bd470409f19a8edd972e16e519d70d0c75adec34c1...
31788322,2020-09-22,f2c314b864a50bf94b0a1e25d9b987293de401930505e5...,909014002,0.118627,1,1.0,1.0,ACTIVE,regularly,6,2c29ae653a9282cce4151bd87643c907644e09541abc28...


In [122]:
tmp2['sigma'] = (tmp2['count_x'] - tmp2['count_y']) / (tmp2['count_x'] + tmp2['count_y']).sqrt()

In [124]:
tmp2[tmp2['sigma']>3]

,article_id,fashion_news_frequency_x,count_x,fashion_news_frequency_y,count_y,sigma
5,854883001,none,369,regularly,279,3.600000
17,608776009,none,1557,regularly,1108,8.803922
27,554479016,none,645,regularly,458,5.666667
34,560222002,none,314,regularly,189,5.681818
37,572127013,none,502,regularly,338,5.857143
...,...,...,...,...,...,...
101428,691254002,none,1003,regularly,748,6.219512
101429,550888001,none,140,regularly,75,4.642857
101430,542464001,none,3539,regularly,2216,17.640000
101435,639685003,none,503,regularly,369,4.620690
